In [2]:
import uproot 
import numpy as np
from tqdm import tqdm
import h5py
import awkward as ak
import vector

In [3]:
path = 'C:/Users/HP/Documents/data/'
pad_to_jet = 20
root_file   = uproot.open(path+'skim_forHyPER_33.root')['Delphes']

In [4]:
root_file.keys()


['Event',
 'Event/Event.Number',
 'Particle',
 'Particle/Particle.PID',
 'Particle/Particle.Mass',
 'Particle/Particle.PT',
 'Particle/Particle.Eta',
 'Particle/Particle.Phi',
 'Jet',
 'Jet/Jet.PT',
 'Jet/Jet.Eta',
 'Jet/Jet.Phi',
 'Jet/Jet.Mass',
 'Jet/Jet.BTag']

In [5]:
root_file['Jet'].array()

<Array [7, 3, 4, 6, 7, 5, 3, 7, ..., 8, 3, 5, 6, 4, 8, 4] type='100000 * int32'>

In [6]:
n_particles = root_file['Particle'].array()
particle_pid = root_file['Particle/Particle.PID'].array()     
particle_m = root_file['Particle/Particle.Mass'].array() 
particle_pt = root_file['Particle/Particle.PT'].array() 
particle_eta = root_file['Particle/Particle.Eta'].array()  
particle_phi = root_file['Particle/Particle.Phi'].array() 
njet = root_file['Jet'].array()                       
jet_pt = root_file['Jet/Jet.PT'].array()  
jet_eta = root_file['Jet/Jet.Eta'].array()   
jet_phi = root_file['Jet/Jet.Phi'].array()     
jet_m = root_file['Jet/Jet.Mass'].array()
jet_btag = root_file['Jet/Jet.BTag'].array()

In [49]:
njet
nbtagged = np.sum(jet_btag, axis=1)
# allMatchedEvent yet to be done
# leastOneMatchedW yet to be done
jet_pt
jet_eta
jet_phi
jet_e = vector.zip({'pt':jet_pt, 'eta':jet_eta, 'phi':jet_phi, 'm' :jet_m}).e
jet_m
jet_btag
# jet_truthmatch yet to be done
particle_pid
particle_pt
particle_eta
particle_phi
particle_m



<Array [[0, 0, 0.33, ..., 0.14, 0.14], ...] type='100000 * var * float32'>

In [14]:
particle_pid[0][np.abs(particle_pid[0]) == 5]

<Array [5, 5, 5, 5, 5, 5, -5, -5, -5, 5, -5, 5] type='12 * int32'>

In [15]:
jet_btag[0]

<Array [0, 0, 0, 0, 0, 0, 1] type='7 * uint32'>

In [216]:
def create_padded_vector(pt, eta, phi, m, max_len, mask=True, pad_eta=0):
    padded_pt = ak.pad_none(particle_pt[mask], max_len, axis=1) #pad each sub array with None to ensure each is the same length 
    padded_pt = ak.fill_none(padded_pt, 0)  #swap the None for 0s
    padded_eta = ak.pad_none(eta[mask], max_len, axis=1)
    padded_eta = ak.fill_none(padded_eta, pad_eta)
    padded_phi = ak.pad_none(phi[mask], max_len, axis=1)
    padded_phi = ak.fill_none(padded_phi, 0)
    padded_m = ak.pad_none(m[mask], max_len, axis=1)
    padded_m = ak.fill_none(padded_m, 0)
    return vector.zip({'pt':padded_pt, 'eta':padded_eta, 'phi':padded_phi, 'm' :padded_m})

In [154]:
max_particles

86

In [421]:
#Finds btagged jets and their corresponding tracks - simple
jet_ids = np.empty((0,len(nbtagged)))
max_btagged = np.max(nbtagged)

particle_pid_reduced = particle_pid[np.abs(particle_pid) == 5]
jet_vectors = create_padded_vector(jet_pt, jet_eta, jet_phi, jet_m, max_btagged, jet_btag == 1, 99)

max_particles = np.max(ak.num(particle_pid_reduced, axis=1))
particle_vectors = vector.zip({'pt':particle_pt, 'eta':particle_eta, 'phi':particle_phi, 'm' :particle_m})[np.abs(particle_pid) == 5]


#padded_particle_pid = ak.pad_none(particle_pid[np.abs(particle_pid) == 5], max_particles, axis=1)
#padded_particle_pid = np.array(ak.fill_none(padded_particle_pid, 99))
#particle_vectors = create_padded_vector(particle_pt, particle_eta, particle_phi, particle_m, max_particles, np.abs(particle_pid) == 5, -99)
for i in range(max_btagged):
    delta_Rs = jet_vectors[:,i].deltaR(particle_vectors)
    # padded_Rs = ak.pad_none(delta_Rs, max_particles, axis=1)
    # padded_Rs = np.array(ak.fill_none(padded_Rs, 99))
    # # delta_Rs is a 2d array with events down the left and different particles on the right
    # # number of particles on right is different for each event
    # # jet_id is the id in the index where the value is lowest
    # jet_id = np.where(np.min(delta_Rs, axis=1)<0.4, ak.Array([row[idx] for row, idx in zip(particle_pid[np.abs(particle_pid)==5], np.argmin(delta_Rs, axis=1))]), 0) # slow line
    # jet_ids = np.vstack((jet_ids, jet_id))
    b = delta_Rs == np.min(delta_Rs, axis=1)[:,np.newaxis]
    jet_id = particle_pid_reduced[b][:,0]
    jet_ids = np.vstack((jet_ids, jet_id))
    # so far don't check if it is closer to this than any of the others






# #Finds btagged jets and their corresponding tracks best 
# btagged_jets = tree["Jet/Jet.BTag"].array()
# is_btagged = tree["Jet/Jet.BTag"].array()==1
# jet_vector_2btagged_mask  = np.sum(btagged_jets, axis=1) == 2
# jet_vectors = vector.zip({'pt': jet_pt, 'eta': jet_eta, 'phi': jet_phi, 'm': jet_mass})[jet_vector_2btagged_mask]
# track_vector = vector.zip({'pt': track_pt, 'eta': track_eta, 'phi': track_phi, 'm': track_m})[jet_vector_2btagged_mask]
# b_vectors = jet_vectors[is_btagged[jet_vector_2btagged_mask]]
# bjet_1=b_vectors[:,0]
# bjet_2=b_vectors[:,1]
# track_vectorb1 = track_vector[(b_vectors[:,0].deltaR(track_vector) < 0.4)&(b_vectors[:,0].deltaR(track_vector) <b_vectors[:,1].deltaR(track_vector)) ]
# track_vectorb2 = track_vector[(b_vectors[:,1].deltaR(track_vector) < 0.4)&(b_vectors[:,1].deltaR(track_vector) <b_vectors[:,0].deltaR(track_vector)) ]

In [424]:
jet_ids

<Array [[-5, 5, -5, 5, ..., 5, -5, -5, -5], ...] type='7 * 100000 * ?float64'>

In [426]:
ak.num(particle_eta[np.abs(particle_pid) == 5], axis=1)[2]

6

In [413]:
a = np.argmin(delta_Rs, axis=1)
a = np.min(delta_Rs, axis=1)
b = delta_Rs == a[:,np.newaxis]

In [414]:
b

<Array [[False, False, ..., False, False], ...] type='100000 * var * ?bool'>

In [417]:
particle_pid_reduced[b][:,0]

<Array [-5, 5, -5, 5, 5, 5, ..., -5, -5, 5, -5, -5, -5] type='100000 * ?int32'>

In [371]:
np.where(np.min(delta_Rs, axis=1)<0.4, padded_particle_pid[np.argmin(padded_Rs)], 0)

IndexError: index 5594042 is out of bounds for axis 0 with size 100000

In [350]:
delta_Rs

<Array [[3.1, 3.02, ..., 0.0952, 2.63], ...] type='100000 * var * float64'>

In [316]:
a = np.argmin(delta_Rs, axis=1)
a

<Array [7, 0, 1, 3, 1, 0, 1, ..., 7, 3, 6, 0, 4, 6, 4] type='100000 * ?int64'>

In [360]:
b = particle_pid[np.abs(particle_pid)==5]


In [358]:
ak.num(delta_Rs)

<Array [12, 10, 6, 9, 7, 7, 6, ..., 6, 10, 9, 7, 11, 9] type='100000 * int64'>

In [354]:
ak.num(particle_vectors, axis=1)

<Array [12, 10, 6, 9, 7, 7, 6, ..., 6, 10, 9, 7, 11, 9] type='100000 * int64'>

In [347]:
ak.Array([row[idx] for row, idx in zip(delta_Rs, a)])

<Array [0.0577, 99.2, 96.1, ..., 0.0285, 0.0316, 0.092] type='100000 * float64'>

In [361]:
[row[idx] for row, idx in zip(b, a)]

[-5,
 5,
 -5,
 5,
 5,
 5,
 5,
 5,
 -5,
 -5,
 -5,
 5,
 -5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 -5,
 5,
 5,
 -5,
 -5,
 -5,
 -5,
 5,
 -5,
 -5,
 -5,
 5,
 -5,
 5,
 -5,
 -5,
 -5,
 5,
 5,
 5,
 5,
 -5,
 5,
 5,
 -5,
 5,
 5,
 5,
 -5,
 -5,
 -5,
 5,
 5,
 5,
 5,
 5,
 -5,
 -5,
 5,
 -5,
 -5,
 -5,
 5,
 -5,
 -5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 -5,
 -5,
 5,
 5,
 -5,
 -5,
 -5,
 5,
 5,
 5,
 5,
 -5,
 5,
 5,
 -5,
 -5,
 -5,
 5,
 -5,
 5,
 -5,
 5,
 -5,
 5,
 5,
 5,
 -5,
 5,
 5,
 -5,
 -5,
 5,
 5,
 -5,
 5,
 -5,
 -5,
 5,
 5,
 -5,
 5,
 -5,
 -5,
 5,
 -5,
 5,
 5,
 -5,
 5,
 -5,
 5,
 5,
 -5,
 5,
 5,
 5,
 -5,
 5,
 -5,
 -5,
 5,
 5,
 5,
 -5,
 -5,
 5,
 5,
 -5,
 -5,
 -5,
 -5,
 -5,
 -5,
 -5,
 5,
 -5,
 5,
 5,
 5,
 5,
 -5,
 -5,
 5,
 -5,
 -5,
 -5,
 -5,
 -5,
 -5,
 5,
 -5,
 -5,
 -5,
 -5,
 5,
 -5,
 5,
 5,
 -5,
 -5,
 5,
 5,
 5,
 -5,
 5,
 5,
 -5,
 -5,
 5,
 -5,
 5,
 -5,
 -5,
 -5,
 5,
 -5,
 -5,
 5,
 -5,
 -5,
 -5,
 5,
 5,
 5,
 5,
 5,
 -5,
 -5,
 -5,
 -5,
 -5,
 -5,
 5,
 -5,
 -5,
 -5,
 -5,
 -5,
 -5,
 -5,
 -5,
 -5,
 -5,
 5,
 5,
 5,
 -5,
 5,
 5,
